- https://www.youtube.com/watch?v=6cM5rmNGGYs
- https://soumilshah1995.blogspot.com/2021/09/pyspark-crash-course-learn-pyspark-in.html

In [1]:
try:
    from pyspark.sql import SparkSession
    from pyspark.sql import SparkSession

    from pyspark.sql.functions import filter
    from pyspark.sql.types import *
    from pyspark.sql.functions import udf
    from pyspark.sql.types import StringType, FloatType, StructType, StructField
    from IPython.core.display import Image, display
    print("all modules are ok ...... ")
    
except Exception as e:
    
    print("some modules are missing : {} ".format(e))

    
spark = SparkSession.builder\
    .appName("MyProcess")\
    .master("local[*]")\
    .getOrCreate()
spark

all modules are ok ...... 
22/10/09 08:24:54 WARN Utils: Your hostname, Daranees-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.20 instead (on interface en0)
22/10/09 08:24:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 08:24:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data   = map(lambda r: (r[0], r[1], float(r[2])),
  map(lambda x: x.split(","),
    ["Paris,Food,19.00", "Marseille,Clothing,12.00",
     "Paris,Food,8.00", "Paris,Clothing,15.00",
     "Marseille,Food,20.00", "Lyon,Book,10.00"]))

In [3]:
schema = StructType([
    StructField("city",  StringType(), nullable=True),
    StructField("type",  StringType(), nullable=True),
    StructField("price", FloatType(),  nullable=True)
])

In [4]:
df = spark.createDataFrame(data, schema=schema)

In [5]:
df.show()

+---------+--------+-----+
|     city|    type|price|
+---------+--------+-----+
|    Paris|    Food| 19.0|
|Marseille|Clothing| 12.0|
|    Paris|    Food|  8.0|
|    Paris|Clothing| 15.0|
|Marseille|    Food| 20.0|
|     Lyon|    Book| 10.0|
+---------+--------+-----+



In [6]:
"""Collect() – Retrieve data from Spark RDD/DataFrame"""
df.collect()

[Row(city='Paris', type='Food', price=19.0),
 Row(city='Marseille', type='Clothing', price=12.0),
 Row(city='Paris', type='Food', price=8.0),
 Row(city='Paris', type='Clothing', price=15.0),
 Row(city='Marseille', type='Food', price=20.0),
 Row(city='Lyon', type='Book', price=10.0)]

In [7]:
"""Returns the first n rows in the DataFrame."""
df.take(4)

[Row(city='Paris', type='Food', price=19.0),
 Row(city='Marseille', type='Clothing', price=12.0),
 Row(city='Paris', type='Food', price=8.0),
 Row(city='Paris', type='Clothing', price=15.0)]

In [8]:
"""gets the first two values """
df.head(2)

[Row(city='Paris', type='Food', price=19.0),
 Row(city='Marseille', type='Clothing', price=12.0)]

In [9]:
"""tails gets the Last values """
df.tail(2)

[Row(city='Marseille', type='Food', price=20.0),
 Row(city='Lyon', type='Book', price=10.0)]

In [10]:
"""selecting single column"""
df.select("City").show(2)

+---------+
|     City|
+---------+
|    Paris|
|Marseille|
+---------+
only showing top 2 rows



In [11]:
"""selecting multiplke column"""
df.select(["city", "type"]).show(2)

+---------+--------+
|     city|    type|
+---------+--------+
|    Paris|    Food|
|Marseille|Clothing|
+---------+--------+
only showing top 2 rows



**filter**

In [12]:
df.filter(df.city == "Paris").show()

+-----+--------+-----+
| city|    type|price|
+-----+--------+-----+
|Paris|    Food| 19.0|
|Paris|    Food|  8.0|
|Paris|Clothing| 15.0|
+-----+--------+-----+



In [13]:
df.filter(df.city == "Paris").filter(df.type == "Food").show()

+-----+----+-----+
| city|type|price|
+-----+----+-----+
|Paris|Food| 19.0|
|Paris|Food|  8.0|
+-----+----+-----+



In [14]:
df.filter(
    (df.city == "Paris") & (df.type == "Food") ).show()

+-----+----+-----+
| city|type|price|
+-----+----+-----+
|Paris|Food| 19.0|
|Paris|Food|  8.0|
+-----+----+-----+



In [15]:
df.filter(
    (df.city == "Paris") & (df.price > 18.0) ).select('city').show()

+-----+
| city|
+-----+
|Paris|
+-----+



In [16]:
df.filter(df.price < 20 ).orderBy(df.price.asc()).select("price").show()

+-----+
|price|
+-----+
|  8.0|
| 10.0|
| 12.0|
| 15.0|
| 19.0|
+-----+



In [17]:
df.filter(df.price < 20 ).sort(df.price.asc()).select("price").show()

+-----+
|price|
+-----+
|  8.0|
| 10.0|
| 12.0|
| 15.0|
| 19.0|
+-----+



In [18]:
df.show(2)

+---------+--------+-----+
|     city|    type|price|
+---------+--------+-----+
|    Paris|    Food| 19.0|
|Marseille|Clothing| 12.0|
+---------+--------+-----+
only showing top 2 rows



**Add New Column**

In [19]:
df.withColumn("50percent", df.price / 2).show()

+---------+--------+-----+---------+
|     city|    type|price|50percent|
+---------+--------+-----+---------+
|    Paris|    Food| 19.0|      9.5|
|Marseille|Clothing| 12.0|      6.0|
|    Paris|    Food|  8.0|      4.0|
|    Paris|Clothing| 15.0|      7.5|
|Marseille|    Food| 20.0|     10.0|
|     Lyon|    Book| 10.0|      5.0|
+---------+--------+-----+---------+



**Drop Column**

In [20]:
df.drop(df.type).show()

+---------+-----+
|     city|price|
+---------+-----+
|    Paris| 19.0|
|Marseille| 12.0|
|    Paris|  8.0|
|    Paris| 15.0|
|Marseille| 20.0|
|     Lyon| 10.0|
+---------+-----+



In [57]:
df.drop(*["city", "price"])\
  .filter(df.type == "Food")\
  .show()

+----+
|type|
+----+
|Food|
|Food|
|Food|
+----+



**rename column(s)**

In [22]:
df.withColumnRenamed("city", "CITY").show()

+---------+--------+-----+
|     CITY|    type|price|
+---------+--------+-----+
|    Paris|    Food| 19.0|
|Marseille|Clothing| 12.0|
|    Paris|    Food|  8.0|
|    Paris|Clothing| 15.0|
|Marseille|    Food| 20.0|
|     Lyon|    Book| 10.0|
+---------+--------+-----+



**Applying Function**

In [62]:
df.dtypes

[('city', 'string'), ('type', 'string'), ('price', 'float')]

In [64]:
@udf("string")
def to_string(price):
    return str(price)

In [70]:
df.select(
    to_string(df.price).alias('PRICE_String')).show()

+------------+
|PRICE_String|
+------------+
|        19.0|
|        12.0|
|         8.0|
|        15.0|
|        20.0|
|        10.0|
+------------+



In [73]:
df_price_string = df.select(to_string(df.price).alias('PRICE_String'))
df_price_string.dtypes

[('PRICE_String', 'string')]

In [74]:
@udf("float")
def to_float(price):
    return float(price)

In [69]:
df.select(
    to_float(df.price).alias('PRICE_Float')).show()

+-----------+
|PRICE_Float|
+-----------+
|       19.0|
|       12.0|
|        8.0|
|       15.0|
|       20.0|
|       10.0|
+-----------+



In [75]:
df_price_float = df.select(to_float(df.price).alias('PRICE_Float'))
df_price_float.dtypes

[('PRICE_Float', 'float')]

In [77]:
# original
df.dtypes

[('city', 'string'), ('type', 'string'), ('price', 'float')]

**Say i want to return JSON**

In [26]:
df.show()

+---------+--------+-----+
|     city|    type|price|
+---------+--------+-----+
|    Paris|    Food| 19.0|
|Marseille|Clothing| 12.0|
|    Paris|    Food|  8.0|
|    Paris|Clothing| 15.0|
|Marseille|    Food| 20.0|
|     Lyon|    Book| 10.0|
+---------+--------+-----+



In [27]:
df.dtypes

[('city', 'string'), ('type', 'string'), ('price', 'float')]

In [84]:
def to_some_transformation(price):
    if price < 15.0:
        return {"name":"value1"}
    else:
        return {"name":"value2"}

In [85]:
udfValueToCategoryGeo = udf(
    to_some_transformation, 
    MapType(StringType(), StringType())
)

In [86]:
df.withColumn("NEW", udfValueToCategoryGeo("price")).show()

+---------+--------+-----+----------------+
|     city|    type|price|             NEW|
+---------+--------+-----+----------------+
|    Paris|    Food| 19.0|{name -> value2}|
|Marseille|Clothing| 12.0|{name -> value1}|
|    Paris|    Food|  8.0|{name -> value1}|
|    Paris|Clothing| 15.0|{name -> value2}|
|Marseille|    Food| 20.0|{name -> value2}|
|     Lyon|    Book| 10.0|{name -> value1}|
+---------+--------+-----+----------------+



**Removing Null and Empty items**

In [87]:
df = spark.createDataFrame([
    [1,'Navee','Srikanth']
    , [2,'','Srikanth'] ,
    [3,'Naveen','']],
    ['ID','FirstName','LastName']
)

df.show()

+---+---------+--------+
| ID|FirstName|LastName|
+---+---------+--------+
|  1|    Navee|Srikanth|
|  2|         |Srikanth|
|  3|   Naveen|        |
+---+---------+--------+



In [88]:
df.na.drop().show()

+---+---------+--------+
| ID|FirstName|LastName|
+---+---------+--------+
|  1|    Navee|Srikanth|
|  2|         |Srikanth|
|  3|   Naveen|        |
+---+---------+--------+



In [89]:
def remove(x):
    if x == '':
        return 1
    else:
        return 0

In [90]:
udfValueToCategory = udf(remove, IntegerType())
df1 = df.withColumn("category", udfValueToCategory("FirstName"))
df1.filter(df1.category == 0).show()

+---+---------+--------+--------+
| ID|FirstName|LastName|category|
+---+---------+--------+--------+
|  1|    Navee|Srikanth|       0|
|  3|   Naveen|        |       0|
+---+---------+--------+--------+



**Aditional Resources**

In [36]:
schema = StructType([
    StructField("city",  StringType(), nullable=True),
    StructField("type",  StringType(), nullable=True),
    StructField("price", FloatType(),  nullable=True)
])

data   = map(lambda r: (r[0], r[1], float(r[2])),
  map(lambda x: x.split(","),
    ["Paris,Food,19.00", "Marseille,Clothing,12.00",
     "Paris,Food,8.00", "Paris,Clothing,15.00",
     "Marseille,Food,20.00", "Lyon,Book,10.00"]))


df = spark.createDataFrame(data, schema=schema)
transactions = df

In [37]:
print('Number of partitions: {}'.format(transactions.rdd.getNumPartitions()))
print('Partitioner: {}'.format(transactions.rdd.partitioner))
print('Partitions structure: {}'.format(transactions.rdd.glom().collect()))

Number of partitions: 8
Partitioner: None
Partitions structure: [[], [Row(city='Paris', type='Food', price=19.0)], [Row(city='Marseille', type='Clothing', price=12.0)], [Row(city='Paris', type='Food', price=8.0)], [], [Row(city='Paris', type='Clothing', price=15.0)], [Row(city='Marseille', type='Food', price=20.0)], [Row(city='Lyon', type='Book', price=10.0)]]


In [93]:
 from IPython.display import Image, display

In [94]:
display(Image(url='https://iamluminousmen-media.s3.amazonaws.com/media/spark-partitions/spark-partitions-2.jpg', width=500, unconfined=True))


- Spark splits data in order to process it in parallel in memory.nce the user has submitted his job into the cluster, each partition is sent to a specific executor for further processing. Only one partition is processed by one executor at a time, so the size and number of partitions transferred to the executor are directly proportional to the time it takes to complete them. Thus the more partitions the more work is distributed to executors, with a smaller number of partitions the work will be done in larger pieces (and often faster).[2]


- The most important reason is performance. By having all the data needed to calculate on a single node, we reduce the overhead on the shuffle (the need for serialization and network traffic).[2]


- The second reason is the cost reduction — better utilization of the cluster will help to reduce idle resources.[2]


What is Differnce between repartition and coalesce ?
- repartition() specifies new number of partition, up or down
- coalesce() uses existing partition, only decrease , no shuffle

In [39]:
df.rdd.saveAsTextFile(path='./learn/test1')

In [40]:
df.rdd.repartition(2).saveAsTextFile(path='./learn/test2')

In [41]:
df.rdd.coalesce(1).saveAsTextFile(path='./learn/test3')

In [42]:
display(Image(url='./1.jpg', width=200, unconfined=True))

In [43]:
df.show()

+---------+--------+-----+
|     city|    type|price|
+---------+--------+-----+
|    Paris|    Food| 19.0|
|Marseille|Clothing| 12.0|
|    Paris|    Food|  8.0|
|    Paris|Clothing| 15.0|
|Marseille|    Food| 20.0|
|     Lyon|    Book| 10.0|
+---------+--------+-----+



**iterating over all columns**

In [44]:
for x in df.rdd.collect():
    print(x.city,end="\n")

Paris
Marseille
Paris
Paris
Marseille
Lyon


In [45]:
df.groupBy("city").mean("price").show()

+---------+----------+
|     city|avg(price)|
+---------+----------+
|    Paris|      14.0|
|Marseille|      16.0|
|     Lyon|      10.0|
+---------+----------+



getting list of unique columns

In [46]:
df.select("city").distinct().show()

+---------+
|     city|
+---------+
|    Paris|
|Marseille|
|     Lyon|
+---------+



In [47]:
"""This is dataframe object """
df

DataFrame[city: string, type: string, price: float]

In [48]:
"""This is RDD object """
df.rdd

MapPartitionsRDD[85] at javaToPython at NativeMethodAccessorImpl.java:0

**Question**
**What is difference betweeb RDD and Dataframe [4]**

- So, a DataFrame has additional metadata due to its tabular format, which allows Spark to run certain optimizations on the finalized query.


- An RDD, on the other hand, is merely a Resilient Distributed Dataset that is more of a blackbox of data that cannot be optimized as the operations that can be performed against it, are not as constrained.


- However, you can go from a DataFrame to an RDD via its rdd method, and you can go from an RDD to a DataFrame (if the RDD is in a tabular format) via the toDF method


- In general it is recommended to use a DataFrame where possible due to the built in query optimization.

**aggregation**

In [49]:
df.groupBy(df.city).agg({"*": "count"}).orderBy("city",ascending=False ).show()

+---------+--------+
|     city|count(1)|
+---------+--------+
|    Paris|       3|
|Marseille|       2|
|     Lyon|       1|
+---------+--------+



References

[1] https://medium.com/@suci/running-pyspark-on-jupyter-notebook-with-docker-602b18ac4494

[2] https://luminousmen.com/post/spark-partitions

[3]https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

[4]https://stackoverflow.com/questions/31508083/difference-between-dataframe-dataset-and-rdd-in-spark

In [95]:
spark.stop()